# Bert

In [1]:
import numpy as np
import pandas as pd
import random as rn
from tqdm.auto import tqdm
import json

from keras.models import Sequential, model_from_json
from keras.layers import *
from keras.optimizers import *
from keras.losses import *

import keras
import tensorflow as tf
import keras.backend as K

In [ ]:
all_data = pd.read_csv('ProcessedTrainData3DB.csv', index_col = 0).dropna()
all_data = all_data[:-1]

In [ ]:
all_data

,Text,Sentiment,Processed Text
0,Ho acquistato questa confezione di mascherine ...,positive,ho acquist confezion mascherin 19 99 quind 40 ...
1,"Attenzione, ha un ferretto nasale in plastica ...",negative,attenzion ferrett nasal plastic permett far ad...
2,"La confezione di cartone era già aperta, appen...",negative,la confezion carton già apert appen pres lat t...
3,"Mascherine vendute come ""Maschera Chirurgica 3...",negative,mascherin vend mascher chirurg 3p tip iir conf...
4,ARRIVATO APERTO PESSIMO PRODOTTO,negative,arriv apert pessim prodott
...,...,...,...
1195,Ho sentito che la signora di Codogno è stata s...,negative,ho sent signor codogn stat sepolt tutt veloc s...
1196,Tutte le università della lombardia rimangono ...,neutral,tutt univers lombard rimang chius fin 29 febbrai
1197,Sei un collega?,neutral,sei colleg
1198,"Ho capito io male, o questo decreto di legge d...",negative,ho cap mal decret legg dic riassum dovrest vie...


In [ ]:
def reducer(all_data = all_data):
  all_data['Processed Text'] = ''
  for index, row in all_data.iterrows():
    l_of_words = all_data.at[index, 'Text']
    all_data.at[index, 'Processed Text'] = " ".join(l_of_words.split(" ")[:100])     
  return      
reducer()

In [ ]:
all_data

,Text,Sentiment,Processed Text
0,Ho acquistato questa confezione di mascherine ...,positive,Ho acquistato questa confezione di mascherine ...
1,"Attenzione, ha un ferretto nasale in plastica ...",negative,"Attenzione, ha un ferretto nasale in plastica ..."
2,"La confezione di cartone era già aperta, appen...",negative,"La confezione di cartone era già aperta, appen..."
3,"Mascherine vendute come ""Maschera Chirurgica 3...",negative,"Mascherine vendute come ""Maschera Chirurgica 3..."
4,ARRIVATO APERTO PESSIMO PRODOTTO,negative,ARRIVATO APERTO PESSIMO PRODOTTO
...,...,...,...
1195,Ho sentito che la signora di Codogno è stata s...,negative,Ho sentito che la signora di Codogno è stata s...
1196,Tutte le università della lombardia rimangono ...,neutral,Tutte le università della lombardia rimangono ...
1197,Sei un collega?,neutral,Sei un collega?
1198,"Ho capito io male, o questo decreto di legge d...",negative,"Ho capito io male, o questo decreto di legge d..."


In [ ]:
def hot_encod(sentiment):
  if (sentiment == "positive"):
    return 1
  elif (sentiment == "negative"):
    return 2
  else:
    return 0

In [ ]:
all_data["Label"] = [hot_encod(x) for x in all_data["Sentiment"]]

In [ ]:
all_data

,Text,Sentiment,Processed Text,Label
0,Ho acquistato questa confezione di mascherine ...,positive,Ho acquistato questa confezione di mascherine ...,1
1,"Attenzione, ha un ferretto nasale in plastica ...",negative,"Attenzione, ha un ferretto nasale in plastica ...",2
2,"La confezione di cartone era già aperta, appen...",negative,"La confezione di cartone era già aperta, appen...",2
3,"Mascherine vendute come ""Maschera Chirurgica 3...",negative,"Mascherine vendute come ""Maschera Chirurgica 3...",2
4,ARRIVATO APERTO PESSIMO PRODOTTO,negative,ARRIVATO APERTO PESSIMO PRODOTTO,2
...,...,...,...,...
1195,Ho sentito che la signora di Codogno è stata s...,negative,Ho sentito che la signora di Codogno è stata s...,2
1196,Tutte le università della lombardia rimangono ...,neutral,Tutte le università della lombardia rimangono ...,0
1197,Sei un collega?,neutral,Sei un collega?,0
1198,"Ho capito io male, o questo decreto di legge d...",negative,"Ho capito io male, o questo decreto di legge d...",2


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-italian-uncased', do_lower_case = True)

Loading BERT tokenizer...


In [ ]:
# Print the original sentence.
print(' Original: ', all_data.loc[0, "Processed Text"])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(all_data.loc[0, "Processed Text"]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(all_data.loc[0, "Processed Text"])))

 Original:  Ho acquistato questa confezione di mascherine a 19.99 quindi 40 centesimi a mascherina, prezzo tutto sommato onesto. La qualità della mascherina è a mio avviso più che discreta, ne ho provate altre e questa ha una consistenza indubbiamente maggiore. Risulta al tatto anche più morbida e spessa. Nella confezione è presente un certificato (cinese) che ne attesta la tipologia IIR. Gli elastici mantengono molto bene, ne ho già utilizzate 5 senza nessun problema. Veniamo ai difetti, essenzialmente due: 1) le mascherine sono chiuse in una scatola ma senza un imballo ad isolarle, nessun plastica di protezione 2) il ferretto che
Tokenized:  ['ho', 'acquistato', 'questa', 'confezione', 'di', 'masch', '##eri', '##ne', 'a', '19', '.', '99', 'quindi', '40', 'centesimi', 'a', 'masch', '##eri', '##na', ',', 'prezzo', 'tutto', 'somma', '##to', 'onesto', '.', 'la', 'qualita', 'della', 'masch', '##eri', '##na', 'e', 'a', 'mio', 'avviso', 'piu', 'che', 'discreta', ',', 'ne', 'ho', 'provate', 

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in all_data["Processed Text"]:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 100,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',
                        truncation = True  # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(all_data["Label"].to_list())

# Print sentence 0, now as a list of IDs.
print('Token IDs:', input_ids[0])

Token IDs: tensor([  102,   332,  8986,   456, 10114,   119,  7558,   277,   306,   111,
          339,   687,  8215,   699,  2913, 21470,   111,  7558,   277,   223,
         1156,  1714,   504,  6701,   112, 15964,   687,   143,  9272,   213,
         7558,   277,   223,   123,   111,   575,  8626,   796,   159, 26851,
         1156,   270,   332, 19699,   938,   123,   456,   248,   214, 19483,
        17484,  1904,   687,  4191,   155, 17939,   370,   796, 20371,   123,
          486,   303,   687,   367, 10114,   123,  1387,   142,  7860,   243,
         5576,  4084,   159,   270, 21373,   143, 10885,  1514, 30939,   687,
          327, 27089, 24149,   577,   460,  1156,   270,   332,   789,  7756,
          533,   779,  2301,  1466,   687, 19440,   377, 15400,  1156,   103])


In [ ]:
#torch.tensor(all_data["Label"])
all_data["Label"]

0       1
1       2
2       2
3       2
4       2
       ..
1195    2
1196    0
1197    0
1198    2
1199    0
Name: Label, Length: 1197, dtype: int64

In [ ]:
input_ids

tensor([[  102,   332,  8986,  ..., 15400,  1156,   103],
        [  102,  3016,  1156,  ...,     0,     0,     0],
        [  102,   143, 10114,  ...,     0,     0,     0],
        ...,
        [  102,   523,   142,  ...,     0,     0,     0],
        [  102,   332,  2472,  ...,     0,     0,     0],
        [  102,  3692,  2809,  ...,     0,     0,     0]])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,077 training samples
  120 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    'dbmdz/bert-base-italian-uncased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (31102, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.93
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.76
  Validation Loss: 0.71
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.60
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.81
  Validation Loss: 0.55
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.43
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 0.79
  Validation Loss: 0.52
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.35
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 0.80
  Validation Loss: 0.51
  Validation took: 0:00:01

Training complete!
Total training took 0:01:20 (h:mm:ss)


In [ ]:
reddit_data = pd.read_csv("redit_data.csv")

In [ ]:
reddit_data

,Unnamed: 0,Author,Submission ID,Text,Hour,Minute,Second,ID,Top Level,Parent ID,#Upvotes,DATE,Processed Text
0,0,Lord_TheJc,f7txjb,"Chiudo il thread, andate su quello nuovo\n\n/r...",12,25,40,fijqzau,True,f7txjb,1,22-2-2020,chiud thread andat nuov r italy comments f87rb...
1,1,EMOSCAMBIO,f7txjb,>Ore 19.30 - Il presunto «paziente 0» negativo...,18,52,58,figab7t,True,f7txjb,83,22-2-2020,ore 19 30 il presunt pazient 0 negat test non ...
2,2,drstins_n,f7txjb,> Si allontana da Codogno e torna ad Avellino:...,21,40,54,fih8gxg,True,f7txjb,77,22-2-2020,si allontan codogn torn avellin famigl quarant...
3,3,IceTea666,f7txjb,Dogana slovenia/croazia. Il poliziotto croato ...,22,17,8,fihft9k,True,f7txjb,75,22-2-2020,dogan sloven croaz il poliziott cro dov contro...
4,4,panoramegamix,f7txjb,ho dato una veloce occhiata sui social \n\nNor...,17,52,2,fifwfbh,True,f7txjb,122,22-2-2020,dat veloc occhi social nord mor centr mor sud ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154651,154651,magicobito,ggqrld,Grazie dello sbattone :),7,47,22,fq92i1z,False,fq7g6n9,1,10-5-2020,graz sbatton
154652,154652,Jeremia19,ggqrld,Tamponi. Fonte: l’han fatto al padre di un mio...,16,41,53,fq6cl9k,False,fq6ci40,1,10-5-2020,tampon font han fatt padr amic cos stat rifer
154653,154653,frost_burg,ggqrld,"Tamponi, mi dicono.",18,39,33,fq6pgpr,False,fq6ci40,1,10-5-2020,tampon dic
154654,154654,Jkal91,ggqrld,"Ma sono le quattro del pomeriggio, che vai a d...",14,25,22,fq5ync4,False,fq5xxtp,4,10-5-2020,ma quattr pomerigg vai dorm 19 ospedal


In [ ]:
def reducer(all_data = all_data):
  all_data['BERT Text'] = ''
  for index, row in all_data.iterrows():
    l_of_words = all_data.at[index, 'Text']
    all_data.at[index, 'BERT Text'] = " ".join(l_of_words.split(" ")[:100])     
  return      
reducer(reddit_data)

In [ ]:
reddit_data

,Unnamed: 0,Author,Submission ID,Text,Hour,Minute,Second,ID,Top Level,Parent ID,#Upvotes,DATE,Processed Text,BERT Text
0,0,Lord_TheJc,f7txjb,"Chiudo il thread, andate su quello nuovo\n\n/r...",12,25,40,fijqzau,True,f7txjb,1,22-2-2020,chiud thread andat nuov r italy comments f87rb...,"Chiudo il thread, andate su quello nuovo\n\n/r..."
1,1,EMOSCAMBIO,f7txjb,>Ore 19.30 - Il presunto «paziente 0» negativo...,18,52,58,figab7t,True,f7txjb,83,22-2-2020,ore 19 30 il presunt pazient 0 negat test non ...,>Ore 19.30 - Il presunto «paziente 0» negativo...
2,2,drstins_n,f7txjb,> Si allontana da Codogno e torna ad Avellino:...,21,40,54,fih8gxg,True,f7txjb,77,22-2-2020,si allontan codogn torn avellin famigl quarant...,> Si allontana da Codogno e torna ad Avellino:...
3,3,IceTea666,f7txjb,Dogana slovenia/croazia. Il poliziotto croato ...,22,17,8,fihft9k,True,f7txjb,75,22-2-2020,dogan sloven croaz il poliziott cro dov contro...,Dogana slovenia/croazia. Il poliziotto croato ...
4,4,panoramegamix,f7txjb,ho dato una veloce occhiata sui social \n\nNor...,17,52,2,fifwfbh,True,f7txjb,122,22-2-2020,dat veloc occhi social nord mor centr mor sud ...,ho dato una veloce occhiata sui social \n\nNor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154651,154651,magicobito,ggqrld,Grazie dello sbattone :),7,47,22,fq92i1z,False,fq7g6n9,1,10-5-2020,graz sbatton,Grazie dello sbattone :)
154652,154652,Jeremia19,ggqrld,Tamponi. Fonte: l’han fatto al padre di un mio...,16,41,53,fq6cl9k,False,fq6ci40,1,10-5-2020,tampon font han fatt padr amic cos stat rifer,Tamponi. Fonte: l’han fatto al padre di un mio...
154653,154653,frost_burg,ggqrld,"Tamponi, mi dicono.",18,39,33,fq6pgpr,False,fq6ci40,1,10-5-2020,tampon dic,"Tamponi, mi dicono."
154654,154654,Jkal91,ggqrld,"Ma sono le quattro del pomeriggio, che vai a d...",14,25,22,fq5ync4,False,fq5xxtp,4,10-5-2020,ma quattr pomerigg vai dorm 19 ospedal,"Ma sono le quattro del pomeriggio, che vai a d..."


In [ ]:
test_inputs = []
test_attention_masks = []

for text in reddit_data["BERT Text"]:
    current_encoded = tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = 100,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = "pt",
        truncation = True
    )
    test_inputs.append(current_encoded["input_ids"])
    test_attention_masks.append(current_encoded["attention_mask"])

input_ids = torch.cat(test_inputs, dim=0)
attention_masks = torch.cat(test_attention_masks, dim=0)

prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=len(input_ids))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

RuntimeError: ignored

In [ ]:
predictions = []
with torch.no_grad():
  for index in tqdm(range(len(b_input_ids) // 1000)):
    outputs = model(b_input_ids[index * 1000 : index * 1000 + 1000], token_type_ids = None, attention_mask=b_input_mask[index * 1000:])
    logits = outputs[0]
    logits = logits.detach().cpu()
    predictions_last.append(logits)

In [ ]:
with torch.no_grad():
  outputs = model(b_input_ids[-656:], token_type_ids=None, attention_mask=b_input_mask[-656:])
  logits = outputs[0]
  logits = logits.detach().cpu()
  predictions.append(logits)

In [ ]:
test_labels = []
for index in tqdm(range(len(predictions))):
  test_labels.extend(list(np.argmax(np.array(predictions[index]), axis = 1)))

In [ ]:
reddit_data["Sentiment"] = test_labels

In [ ]:
reddit_data.to_csv("reddit_with_sentiment.csv")